In [1]:
from openai import OpenAI
from pinecone import Pinecone
import os
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from collections import defaultdict
from IPython.display import display, clear_output
from textwrap import wrap


c:\Users\anees\Desktop\Coding\llmrepo\train-LLAMA\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pc = Pinecone(os.getenv("PINECONE_KEY"))
index = pc.Index("resume-index-full")

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'
)

In [13]:
def LLM_chatbot(user_messsage, conversation_history):
    try:
        conversation_history.append(user_messsage)
        response_stream = client.chat.completions.create(
            model="mistral:latest",
            messages=conversation_history,
            stream=True
        )

        output_box = "" 
        for chunk in response_stream:
            if chunk.choices and len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if delta and delta.content:
                    output_box += delta.content
                    display(output_box)  
                    clear_output(wait=True) 

        conversation_history.append({"role": "assistant", "content": output_box})

    except Exception as e:
        print(e)


In [6]:
def query_to_chat(query):
    # Encode query and retrieve relevant chunks as before
    embedded_query = model.encode(query).tolist()
    retrieved_chunks = index.query(
        vector=embedded_query,
        top_k=5,
        include_metadata=True
    )

    dic = defaultdict(list)
    for match in retrieved_chunks.matches:
        section = match.metadata.get('section', 'Unknown Section')
        text = match.metadata.get('text', '')
        dic[section].append(text)


    chunks_text = ""
    for section, texts in dic.items():
        chunks_text += f"{section}:\n"
        for text in texts:
            chunks_text += f"{text}\n\n"

    prompt_header = (
        "You are Chotu the resume assistant chatbot for Aneesh Patne. "
        "You will be asked a question based on the resume data. "
        "You must provide a detailed, relevant, and professional response strictly based on the resume information. "
        "you shall not mention from where you got the information. "
        "If the question cannot be answered using the resume data, state that you can only answer questions related to the resume.\n\n"
        "You are a resume assistant chatbot for Aneesh Patne. Using ONLY the following information from the resume, answer the question below:\n\n"
    )
    prompt_footer = (
        f"Question: \"{query}\"\n\n"
        "Provide a detailed, relevant, and professional response strictly based on the resume information above. "
        "If the question cannot be answered using the resume data, state that you can only answer questions related to the resume."
        "You will in no way provide help which is not related to the resume"
        "You are free to make any assumptions based on the resume data. "
        "If the questions is about the skills, you have to keep a positive tone and say yes but it would be better if you contact Aneesh Patne for more information. "
        "if the question is open-ended based on the resume data, you are free to answer it in your own words. and tell the user to contact Aneesh Patne for more information. "
        "you will politely state that you can only respond based on the resume."
    )
    prompt = prompt_header + chunks_text + prompt_footer

    user_message = {"role": "user", "content": prompt}

    
    conversation_history = [{
        "role": "system",
        "content": (
            "You are a professional Resume Assistant bot for Aneesh Patne. "
            "Your entire purpose is to answer questions strictly based on Aneesh Patne's resume data. "
            "Do not generate answers, instructions, or advice that are not directly supported by the resume information provided. "
            "If a question falls outside the scope of the resume, politely state that you can only respond based on the resume."
        )
    }]

    LLM_chatbot(user_message, conversation_history)


In [14]:
query = "Are you resume assistant for Aneesh Patne?"
query_to_chat(query)

" Yes, I am a Resume Assistant specifically for Aneesh Patne. From his resume, it is clear that he is an experienced Electronics and Telecommunication professional with expertise in areas such as machine learning, web development, cloud platforms, and more. His recent project involved developing a chatbot utilizing Retrieval-Augmented Generation (RAG) for accurate and context-specific responses during March 2023 to June 2023. Additionally, he has been certified as a Machine Learning Specialist by Coursera. I would be pleased to help you learn more about Aneesh Patne's experience, skills, or expertise; however, for a more detailed discussion or conversation, it is best to reach out directly to him through his provided contact information:\n\nLinkedIn - [https://www.linkedin.com/in/aneeshpatne](https://www.linkedin.com/in/aneeshpatne)\nE-mail - aneeshpatne12@gmail.com\nGitHub  - [https://github.com/aneeshpatne](https://github.com/aneeshpatne)\nLeetCode  - [https://leetcode.com/aneeshpatn